In [1]:
from glob import glob
import numpy as np
import math
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from astropy.io import fits
from scripts.visualization_helpers import *
import jwst
print(jwst.__version__)

1.10.2


In [1]:
1e+3*610*2

1220000.0

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib as mpl
%matplotlib inline

# figure sizes and resolutions
rcParams['figure.figsize'] = [10,10]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

cmap = plt.get_cmap('summer')
cmaplist = [cmap(i) for i in range(cmap.N)]

# create the new map
cmap = cmap.from_list('Custom cmap', cmaplist[0:], cmap.N)


In [3]:
pids = [pid.split('/')[-1] for pid in glob.glob('/data/scratch/bariskurtkaya/dataset/NIRCAM/*')]
pids

['1550',
 '1184',
 '1412',
 '1189',
 '4454',
 '1194',
 '1411',
 '1482',
 '4498',
 '3989',
 '1193',
 '4558',
 '1386',
 '4497',
 '1563',
 '4452',
 '1075',
 '4050',
 '2780',
 '1441',
 '2278',
 '3840',
 '4451']

In [11]:
np.sqrt(fits.open(glob.glob('/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/*psfstack.fits')[0])[1].header['PIXAR_A2'])

0.06259530358142339

In [ ]:
for pid in sorted(pids):
    print(f"{pid}")
    for suffix in ['rateints','calints','crfints','psfstack','psfsub','i2d']:
        directory           = f'/data/scratch/bariskurtkaya/dataset/NIRCAM/{pid}/mastDownload/JWST/'
        visualizer_psfstack = FITSVisualizer(directory=directory, suffix=suffix)
        products = visualizer_psfstack.get_products()
    
    print("######################################")

    

In [ ]:
PROPOSAL_ID = '1412'
directory = f'/data/scratch/bariskurtkaya/dataset/NIRCAM/{PROPOSAL_ID}/mastDownload/JWST/'

In [ ]:
fits_visualizer = FITSVisualizer(directory=directory, suffix='psfstack')

In [ ]:
things = fits_visualizer.get_things()

In [ ]:
things['sci_array'][0].shape

In [ ]:
fits_visualizer.plot()